<a href="https://colab.research.google.com/github/cs20m016/cs6910-Assignment-1/blob/main/Que2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [4]:
from keras.datasets import fashion_mnist

In [15]:
(train_X,train_Y), (test_X,test_Y)= fashion_mnist.load_data()

In [16]:
train_X.shape, train_Y.shape

((60000, 28, 28), (60000,))

In [17]:
# Preparing the data for training

In [18]:
train_X = train_X[:]/255
test_Y = test_Y[:]/255

In [19]:
train_X=[train_X[i].flatten() for i in range(len(train_X))]
test_X=[test_X[i].flatten() for i in range(len(test_X))]

In [20]:
np.shape(train_X), np.shape(train_Y), np.shape(test_X), np.shape(test_Y)

((60000, 784), (60000,), (10000, 784), (10000,))

In [86]:
#N = number of inputs
N = 3
#L = number of layers
L = 3
#H = number of hidden layers
Hid = L-1

In [63]:
def sigmoid(a):
  h = []
  for a1 in a:
    h.append(1.0/(1.0 + np.exp(-a1)))
  return h

In [64]:
def softmax(a):
  y = []
  sum = 0
  for a1 in a:
    sum+= np.exp(a1)
  for a1 in a:
    y.append(np.exp(a1)/sum)
  return y

In [87]:
def initialize_weights(n_input, n_hidden_layer, n_output,neurons_hl):
    W = list()
    W.append([[random.random() for j in range(n_input)]for i in range(neurons_hl)])
    for i in range(n_hidden_layer-1):
        W.append([[random.random() for j in range(neurons_hl)]for i in range(neurons_hl)])
    W.append([[random.random() for j in range(neurons_hl)]for i in range(n_output)])
    return W
W=initialize_weights(len(train_X[0]),Hid,10,N)

In [27]:
len(W)

3

In [60]:
def initialize_bias(n_input, n_hidden_layer, n_output,n_neurons):
    b=list()
    for i in range(n_hidden_layer):
        b.append([random.random() for i in range(n_neurons)])
    b.append([random.random() for i in range(n_output)])
    return b
b = initialize_bias(len(train_X[0]),L-1,10,N)

In [38]:
len(b)

3

In [44]:
b

[[0.9327478092021355, 0.6627149528718662, 0.43753020348960503],
 [0.16234925170105974, 0.2255810667020044, 0.7838614426209409],
 [0.8850178144595041,
  0.033263924031330716,
  0.1469846924592222,
  0.29985406115181223,
  0.4811324235841644,
  0.4861737829149505,
  0.09942125383087941,
  0.24784414961952073,
  0.843271992046654,
  0.8803901673473634]]

In [37]:
np.shape(b[0]), np.shape(b[2])

((3,), (10,))

In [104]:
def feed_forward_prop(input,W,b,L):
  H=[]
  A=[]
  h=input
  H.append(h)
  A.append([0])
  for i in range(1,L):
      a=b[i-1]+np.matmul(W[i-1],h)
      A.append(a)
      h=sigmoid(a)
      H.append(h)
  aL=b[L-1]+np.matmul(W[L-1],h)
  A.append(aL)
  hL=softmax(aL)
  H.append(hL)
  return hL

In [100]:
train_X[0].shape

(784,)

In [110]:
Y_pred = list()
for x in train_X:
  y_hat = feed_forward_prop(x,W,b,L)
  Y_pred.append(y_hat)